In [9]:
# ============================================================
# CareTrace mini KG-RAG scaffold (Ollama >= 0.15.x)
# Methods:
#   1) KG-augmented retrieval
#   2) KG-augmented reasoning (iterative multi-hop)
#   3) Controlled generation (rule/checklist constrained output)
#
# Requirements:
#   pip install requests
#   Ollama running locally (you already have it)
#   A model pulled, e.g.:
#       ollama pull llama3.1
#       ollama pull qwen2.5:3b
#
# ============================================================

import json
import re
import time
from dataclasses import dataclass
from typing import Dict, List, Tuple, Any, Optional
import requests

In [10]:
#Ollama check

print(requests.get("http://localhost:11434/api/tags").json()["models"][:5])

[{'name': 'llama3.1:latest', 'model': 'llama3.1:latest', 'modified_at': '2026-02-01T22:34:53.017843659-08:00', 'size': 4920753328, 'digest': '46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '8.0B', 'quantization_level': 'Q4_K_M'}}, {'name': 'deepseek-r1:8b', 'model': 'deepseek-r1:8b', 'modified_at': '2025-12-15T19:35:01.922019019-08:00', 'size': 5225376047, 'digest': '6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'qwen3', 'families': ['qwen3'], 'parameter_size': '8.2B', 'quantization_level': 'Q4_K_M'}}, {'name': 'mistral:latest', 'model': 'mistral:latest', 'modified_at': '2025-12-15T09:23:38.892044999-08:00', 'size': 4372824384, 'digest': '6577803aa9a036369e481d648a2baebb381ebc6e897f2bb9a766a2aa7bfbc1cf', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'famil

In [11]:
# -----------------------------
# 0) Ollama client (0.15.x)
# -----------------------------
class OllamaClient:
    def __init__(self, model="llama3.1", base_url="http://localhost:11434"):
        self.model = model
        self.base_url = base_url.rstrip("/")

    def generate(self, prompt: str, temperature: float = 0.2, timeout: int = 120) -> str:
        url = f"{self.base_url}/api/generate"
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": temperature},
        }
        r = requests.post(url, json=payload, timeout=timeout)
        r.raise_for_status()
        return r.json().get("response", "")


In [12]:
# -----------------------------
# 1) A small CareTrace knowledge graph (JSON-friendly)
# -----------------------------
# This is NOT SNOMED-complete. It's a scoped KG for this example:
# symptoms, conditions, red-flags, actions, and "why" relationships.
# The goal is to show *how* KG evidence changes LLM behavior.

CARETRACE_KG = {
    "nodes": [
        # Symptoms / Observations
        {"id": "sym_fever", "type": "Symptom", "name": "Fever", "aliases": ["fever", "temperature", "hot"]},
        {"id": "sym_vomit", "type": "Symptom", "name": "Vomiting", "aliases": ["vomit", "threw up", "throw up", "emesis"]},
        {"id": "sym_abdpain", "type": "Symptom", "name": "Abdominal pain", "aliases": ["stomach hurts", "belly pain", "tummy pain"]},
        {"id": "sym_lethargy", "type": "Symptom", "name": "Lethargy", "aliases": ["wiped out", "very tired", "hard to wake", "sleepy"]},
        {"id": "sym_lowurine", "type": "Finding", "name": "Low urine output", "aliases": ["only peed once", "not peeing", "no pee"]},
        {"id": "sym_blue_lips", "type": "Finding", "name": "Blue lips", "aliases": ["blue lips", "bluish lips", "turning blue"]},
        {"id": "sym_breathing", "type": "Finding", "name": "Trouble breathing", "aliases": ["hard to breathe", "struggling to breathe", "labored breathing"]},

        # Conditions (scoped)
        {"id": "cond_viral_gi", "type": "Condition", "name": "Viral gastroenteritis", "aliases": ["stomach bug", "viral GI"]},
        {"id": "cond_dehydration", "type": "Condition", "name": "Dehydration", "aliases": ["dehydrated"]},
        {"id": "cond_appendicitis", "type": "Condition", "name": "Appendicitis", "aliases": ["appendicitis"]},
        {"id": "cond_respiratory_distress", "type": "Condition", "name": "Respiratory distress", "aliases": ["resp distress"]},

        # Actions / Plans
        {"id": "act_ors", "type": "Action", "name": "Oral rehydration (ORS)", "aliases": ["pedialyte", "oral rehydration solution"]},
        {"id": "act_small_sips", "type": "Action", "name": "Small frequent sips", "aliases": ["small sips", "tiny sips", "sip frequently"]},
        {"id": "act_tylenol", "type": "Action", "name": "Acetaminophen (Tylenol)", "aliases": ["acetaminophen", "tylenol"]},
        {"id": "act_ibuprofen", "type": "Action", "name": "Ibuprofen (Motrin)", "aliases": ["ibuprofen", "motrin", "advil"]},
        {"id": "act_monitor", "type": "Action", "name": "Monitor overnight", "aliases": ["monitor", "check every few hours"]},
        {"id": "act_escalate_er", "type": "Action", "name": "Escalate to ER now", "aliases": ["go to ER", "ER now", "emergency"]},
        {"id": "act_call_peds", "type": "Action", "name": "Call pediatrician in morning", "aliases": ["call pediatrician", "call doctor"]},

        # Rules / Thresholds as nodes (so retrieval can pull them)
        {"id": "rule_no_urine_8h", "type": "Rule", "name": "No urine for ~8 hours is urgent", "aliases": ["no pee 8 hours", "no urine 8 hours"]},
        {"id": "rule_green_vomit", "type": "Rule", "name": "Green (bilious) vomit is an emergency", "aliases": ["green vomit", "bilious"]},
        {"id": "rule_rlq_pain", "type": "Rule", "name": "Worsening localized RLQ pain suggests appendicitis", "aliases": ["right lower", "rlq"]},
        {"id": "rule_blue_lips", "type": "Rule", "name": "Blue lips + breathing trouble is emergency", "aliases": ["blue lips emergency"]},
        {"id": "rule_ibuprofen_dehydration", "type": "Rule", "name": "Avoid ibuprofen if dehydrated/low urine", "aliases": ["avoid ibuprofen dehydration"]},
    ],
    "edges": [
        # Symptom -> suggests -> condition
        {"src": "sym_lowurine", "rel": "SUGGESTS", "dst": "cond_dehydration", "why": "Low urine output can indicate dehydration."},
        {"src": "sym_vomit", "rel": "SUGGESTS", "dst": "cond_viral_gi", "why": "Vomiting can occur in viral gastroenteritis."},
        {"src": "sym_abdpain", "rel": "SUGGESTS", "dst": "cond_appendicitis", "why": "Abdominal pain can be appendicitis depending on pattern."},
        {"src": "sym_blue_lips", "rel": "SUGGESTS", "dst": "cond_respiratory_distress", "why": "Cyanosis suggests inadequate oxygenation."},
        {"src": "sym_breathing", "rel": "SUGGESTS", "dst": "cond_respiratory_distress", "why": "Work of breathing signals respiratory distress."},

        # Condition -> recommend -> action
        {"src": "cond_dehydration", "rel": "RECOMMENDS", "dst": "act_ors", "why": "ORS replaces fluids and electrolytes."},
        {"src": "cond_dehydration", "rel": "RECOMMENDS", "dst": "act_small_sips", "why": "Small sips reduce vomiting risk."},

        # Symptom -> recommend -> action
        {"src": "sym_fever", "rel": "RECOMMENDS", "dst": "act_tylenol", "why": "Acetaminophen helps discomfort/fever."},
        {"src": "sym_fever", "rel": "RECOMMENDS", "dst": "act_ibuprofen", "why": "Ibuprofen helps fever/aches but has cautions."},

        # Rules -> constrain -> action
        {"src": "rule_ibuprofen_dehydration", "rel": "CONSTRAINS", "dst": "act_ibuprofen", "why": "Dehydration increases kidney risk with NSAIDs."},
        {"src": "rule_no_urine_8h", "rel": "TRIGGERS", "dst": "act_escalate_er", "why": "Very low urine output can indicate significant dehydration."},
        {"src": "rule_green_vomit", "rel": "TRIGGERS", "dst": "act_escalate_er", "why": "Bilious vomiting can indicate obstruction."},
        {"src": "rule_rlq_pain", "rel": "TRIGGERS", "dst": "act_escalate_er", "why": "Appendicitis needs urgent evaluation."},
        {"src": "rule_blue_lips", "rel": "TRIGGERS", "dst": "act_escalate_er", "why": "Cyanosis with breathing trouble is emergency."},

        # Monitoring / follow-up
        {"src": "cond_viral_gi", "rel": "RECOMMENDS", "dst": "act_monitor", "why": "Most viral GI improves with hydration/support."},
        {"src": "act_monitor", "rel": "FOLLOWED_BY", "dst": "act_call_peds", "why": "If stable overnight, follow up in the morning."},
    ],
}

In [13]:
# -----------------------------
# 2) KG utility: simple retrieval
# -----------------------------
@dataclass
class Node:
    id: str
    type: str
    name: str
    aliases: List[str]

@dataclass
class Edge:
    src: str
    rel: str
    dst: str
    why: str

class SimpleKG:
    def __init__(self, kg_json: Dict[str, Any]):
        self.nodes: Dict[str, Node] = {}
        for n in kg_json["nodes"]:
            self.nodes[n["id"]] = Node(
                id=n["id"],
                type=n["type"],
                name=n["name"],
                aliases=n.get("aliases", []),
            )
        self.edges: List[Edge] = [Edge(**e) for e in kg_json["edges"]]

        # Build inverted index: keyword -> node_id
        self.term_index: Dict[str, List[str]] = {}
        for node in self.nodes.values():
            for term in [node.name] + node.aliases:
                for tok in self._tokenize(term):
                    self.term_index.setdefault(tok, []).append(node.id)

    def _tokenize(self, text: str) -> List[str]:
        text = text.lower()
        text = re.sub(r"[^a-z0-9\s]", " ", text)
        toks = [t for t in text.split() if len(t) >= 3]
        return toks

    def find_nodes_in_text(self, text: str) -> List[str]:
        toks = self._tokenize(text)
        hits = []
        for t in toks:
            hits.extend(self.term_index.get(t, []))
        # unique preserve order
        seen = set()
        out = []
        for h in hits:
            if h not in seen:
                seen.add(h)
                out.append(h)
        return out

    def retrieve_relevant_edges(self, query: str, max_edges: int = 14) -> List[Edge]:
        # naive: find nodes mentioned, return edges touching those nodes
        node_ids = self.find_nodes_in_text(query)
        if not node_ids:
            return []

        edges = []
        for e in self.edges:
            if e.src in node_ids or e.dst in node_ids:
                edges.append(e)

        # If too few, expand one hop: include edges connected to any dst of selected edges
        if len(edges) < max_edges:
            expanded = set(node_ids)
            for e in edges:
                expanded.add(e.src)
                expanded.add(e.dst)
            for e in self.edges:
                if (e.src in expanded or e.dst in expanded) and e not in edges:
                    edges.append(e)

        return edges[:max_edges]

    def edges_to_text(self, edges: List[Edge]) -> str:
        lines = []
        for e in edges:
            s = self.nodes[e.src].name if e.src in self.nodes else e.src
            d = self.nodes[e.dst].name if e.dst in self.nodes else e.dst
            lines.append(f"- ({self.nodes[e.src].type}) {s}  --{e.rel}-->  ({self.nodes[e.dst].type}) {d} :: {e.why}")
        return "\n".join(lines)

    def get_rules(self) -> List[Node]:
        return [n for n in self.nodes.values() if n.type == "Rule"]

In [14]:
# -----------------------------
# 3) Method 1: KG-augmented retrieval
# -----------------------------

def kg_augmented_retrieval(llm: OllamaClient, kg: SimpleKG, user_question: str) -> str:
    edges = kg.retrieve_relevant_edges(user_question)
    evidence = kg.edges_to_text(edges) if edges else "(no relevant KG evidence found)"

    prompt = f"""
You are a careful after-hours pediatric triage assistant.

Use ONLY the evidence below to answer.
If the evidence is insufficient, say so and ask 1-2 targeted follow-up questions.

Evidence:
{evidence}

User message:
{user_question}

Write:
- 6 to 10 concise bullet points
- Include what to do tonight and what to monitor
- Include clear escalation thresholds if applicable
"""
    return llm.generate(prompt, temperature=0.2)


In [15]:
# -----------------------------
# 4) Method 2: KG-augmented reasoning (iterative multi-hop)
# -----------------------------
def extract_candidate_followups(text: str) -> List[str]:
    # Heuristic: pull lines that look like questions; if none, suggest typical triage questions.
    qs = []
    for line in text.splitlines():
        line = line.strip()
        if line.endswith("?"):
            qs.append(line)
    if qs:
        return qs[:2]
    return [
        "How long has it been since the last urination (hours)?",
        "Is the belly pain localized (especially right-lower), and is it getting worse?",
    ]


def kg_augmented_reasoning(
    llm: OllamaClient,
    kg: SimpleKG,
    conversation: List[str],
    max_steps: int = 3
) -> Dict[str, Any]:
    """
    Iteratively:
      - summarize current state
      - retrieve KG edges based on state + last message
      - ask/decide next action
    """
    state = {
        "conversation": conversation[:],
        "steps": [],
        "final_answer": None
    }

    for step in range(1, max_steps + 1):
        last = state["conversation"][-1]
        context = "\n".join([f"- {m}" for m in state["conversation"]])

        edges = kg.retrieve_relevant_edges(context, max_edges=18)
        evidence = kg.edges_to_text(edges) if edges else "(no relevant KG evidence found)"

        prompt = f"""
You are a careful after-hours pediatric triage assistant.

Conversation so far:
{context}

Evidence from a clinical knowledge graph:
{evidence}

Task:
1) Give an updated plan (5-8 bullets) based on what is known.
2) If critical info is missing to decide safely, ask at most 2 follow-up questions.
3) Include escalation thresholds if any are triggered by the evidence.

Be concise and remain consistent with earlier steps.
"""
        out = llm.generate(prompt, temperature=0.2)
        followups = extract_candidate_followups(out)

        state["steps"].append({
            "step": step,
            "evidence_edges": len(edges),
            "model_output": out,
            "suggested_followups": followups
        })

        # Stopping heuristic: if output contains "ER" triggers or gives a stable plan without questions
        if ("?" not in out) or ("go to the er" in out.lower()) or ("er now" in out.lower()):
            state["final_answer"] = out
            break

        # If still asking questions, simulate one additional user answer for demo purposes
        state["conversation"].append("(demo) last pee was ~6 hours ago; sipping fluids; belly pain diffuse not localized.")

    if state["final_answer"] is None:
        state["final_answer"] = state["steps"][-1]["model_output"] if state["steps"] else ""

    return state

In [16]:
# -----------------------------
# 5) Method 3: Controlled generation
# -----------------------------
# Key idea: we don't let the LLM "freewheel".
# We provide a tight template + a rules checklist + require specific sections.

CONTROLLED_TEMPLATE = """
You are CareTrace, an after-hours pediatric triage assistant.

You must output EXACTLY these sections, in order, with the labels exactly as shown:

1) What I think is going on (1-2 sentences)
2) What to do tonight (3-6 bullets)
3) What I need you to watch for (3-6 bullets)
4) Go now if ANY of these happen (3-6 bullets)

Constraints:
- Do not add any other sections.
- Do not provide long explanations.
- Do not provide medication dosing unless weight AND concentration are provided.
- Use only the evidence and rules below.
"""

def controlled_generation(llm: OllamaClient, kg: SimpleKG, user_question: str) -> str:
    edges = kg.retrieve_relevant_edges(user_question, max_edges=22)
    evidence = kg.edges_to_text(edges) if edges else "(no relevant KG evidence found)"

    # Pull "Rule" nodes as a compact checklist
    rules = kg.get_rules()
    rules_text = "\n".join([f"- {r.name}" for r in rules]) if rules else "(no rules)"

    prompt = f"""
{CONTROLLED_TEMPLATE}

Evidence:
{evidence}

Rules checklist:
{rules_text}

User message:
{user_question}
"""
    return llm.generate(prompt, temperature=0.2)

In [17]:
# -----------------------------
# 6) Main
# -----------------------------

def main():
    # Pick a fast small model if you want:
    # llm = OllamaClient(model="qwen2.5:3b")
    llm = OllamaClient(model="llama3.1")

    # Quick connectivity test
    print("=== Ollama connectivity test ===")
    try:
        print(llm.generate("Return exactly: OK", temperature=0.0).strip())
    except Exception as e:
        print("Ollama call failed. Check that Ollama is running and model is pulled.")
        raise

    kg = SimpleKG(CARETRACE_KG)

    # Three questions that map respectively to the three methods
    q1 = "It’s 11 PM. My 6-year-old has a fever, vomited once, stomach hurts, and he’s only urinated once today. What should I do tonight?"
    q2 = "He has fever and belly pain but can sip fluids. I want to avoid the ER if possible. What should I watch for?"
    q3 = "He has trouble breathing and looks blue around the lips."

    print("\n=== Method 1: KG-augmented retrieval ===")
    print(kg_augmented_retrieval(llm, kg, q1))

    print("\n=== Method 2: KG-augmented reasoning (iterative) ===")
    out = kg_augmented_reasoning(llm, kg, conversation=[q1, q2], max_steps=3)
    print("Final answer:\n", out["final_answer"])
    print("\n(Reasoning trace steps:", len(out["steps"]), ")")

    print("\n=== Method 3: Controlled generation ===")
    print(controlled_generation(llm, kg, q3))

    # Optional: show the KG evidence pulled for a query
    print("\n=== Debug: Evidence retrieved for q1 ===")
    edges = kg.retrieve_relevant_edges(q1, max_edges=18)
    print(kg.edges_to_text(edges))

if __name__ == "__main__":
    main()


=== Ollama connectivity test ===
OK

=== Method 1: KG-augmented retrieval ===
Here are the recommendations based on the evidence:

**Tonight:**

* Monitor your child's urine output closely.
* Encourage small, frequent sips of fluids to reduce vomiting risk.
* Avoid giving ibuprofen (Motrin) as it may increase kidney risk with dehydration.
* Consider oral rehydration (ORS) if your child is showing signs of dehydration.

**Monitoring tonight:**

* Keep an eye on urine output and report any decrease or absence of urination.
* Monitor for signs of dehydration, such as dry mouth, sunken eyes, or decreased tears.
* Watch for changes in vomiting frequency or severity.

**Escalation thresholds:**

* If your child's urine output decreases significantly (e.g., no urination for 4-6 hours) or shows other signs of severe dehydration, call the pediatrician immediately.
* If your child's condition worsens or you have concerns about their safety, call the pediatrician at any time.

**Additional guidan

In [18]:
# ============================================================
# VERBOSE CareTrace KG-RAG lab scaffold (shows intermediate steps)
# Adds:
#   - Verbose iterative reasoning: prints evidence + plan refinement each step
#   - Controlled generation with a "Cypher-style" retrieval step:
#       * Option A: run against Neo4j AuraDB (if you provide URI/user/pw)
#       * Option B: fallback to in-memory KG retrieval (same as before)
#
# Requirements:
#   pip install requests
#   (Optional for Neo4j) pip install neo4j
# ============================================================




# -----------------------------
# 0) Ollama client (Ollama >= 0.15.x)
# -----------------------------
class OllamaClient:
    def __init__(self, model="llama3.1", base_url="http://localhost:11434"):
        self.model = model
        self.base_url = base_url.rstrip("/")

    def generate(self, prompt: str, temperature: float = 0.2, timeout: int = 120) -> str:
        url = f"{self.base_url}/api/generate"
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": temperature},
        }
        r = requests.post(url, json=payload, timeout=timeout)
        r.raise_for_status()
        return r.json().get("response", "")


# -----------------------------
# 2) Simple KG retrieval utilities
# -----------------------------
@dataclass
class Node:
    id: str
    type: str
    name: str
    aliases: List[str]

@dataclass
class Edge:
    src: str
    rel: str
    dst: str
    why: str

class SimpleKG:
    def __init__(self, kg_json: Dict[str, Any]):
        self.nodes: Dict[str, Node] = {}
        for n in kg_json["nodes"]:
            self.nodes[n["id"]] = Node(
                id=n["id"],
                type=n["type"],
                name=n["name"],
                aliases=n.get("aliases", []),
            )
        self.edges: List[Edge] = [Edge(**e) for e in kg_json["edges"]]

        self.term_index: Dict[str, List[str]] = {}
        for node in self.nodes.values():
            for term in [node.name] + node.aliases:
                for tok in self._tokenize(term):
                    self.term_index.setdefault(tok, []).append(node.id)

    def _tokenize(self, text: str) -> List[str]:
        text = text.lower()
        text = re.sub(r"[^a-z0-9\s]", " ", text)
        toks = [t for t in text.split() if len(t) >= 3]
        return toks

    def find_nodes_in_text(self, text: str) -> List[str]:
        toks = self._tokenize(text)
        hits = []
        for t in toks:
            hits.extend(self.term_index.get(t, []))
        seen = set()
        out = []
        for h in hits:
            if h not in seen:
                seen.add(h)
                out.append(h)
        return out

    def retrieve_relevant_edges(self, query: str, max_edges: int = 14) -> List[Edge]:
        node_ids = self.find_nodes_in_text(query)
        if not node_ids:
            return []

        edges = []
        for e in self.edges:
            if e.src in node_ids or e.dst in node_ids:
                edges.append(e)

        if len(edges) < max_edges:
            expanded = set(node_ids)
            for e in edges:
                expanded.add(e.src)
                expanded.add(e.dst)
            for e in self.edges:
                if (e.src in expanded or e.dst in expanded) and e not in edges:
                    edges.append(e)

        return edges[:max_edges]

    def edges_to_text(self, edges: List[Edge]) -> str:
        lines = []
        for e in edges:
            s = self.nodes[e.src].name if e.src in self.nodes else e.src
            d = self.nodes[e.dst].name if e.dst in self.nodes else e.dst
            lines.append(f"- ({self.nodes[e.src].type}) {s}  --{e.rel}-->  ({self.nodes[e.dst].type}) {d} :: {e.why}")
        return "\n".join(lines)

    def get_rules(self) -> List[Node]:
        return [n for n in self.nodes.values() if n.type == "Rule"]


# -----------------------------
# 3) Verbose method 2: iterative reasoning you can watch refine
# -----------------------------
def verbose_kg_augmented_reasoning(
    llm: OllamaClient,
    kg: SimpleKG,
    conversation: List[str],
    max_steps: int = 3,
    print_evidence: bool = True
) -> Dict[str, Any]:
    """
    Shows per-step:
      - retrieved evidence
      - draft plan
      - follow-up questions
      - simulated (or real) next user info
    """
    state = {
        "conversation": conversation[:],
        "steps": [],
        "final_answer": None
    }

    for step in range(1, max_steps + 1):
        context = "\n".join([f"- {m}" for m in state["conversation"]])
        edges = kg.retrieve_relevant_edges(context, max_edges=18)
        evidence = kg.edges_to_text(edges) if edges else "(no relevant KG evidence found)"

        if print_evidence:
            print("\n" + "=" * 70)
            print(f"STEP {step}: conversation context")
            print(context)
            print("\nSTEP {step}: retrieved KG evidence")
            print(evidence)

        prompt = f"""
You are CareTrace, an after-hours pediatric triage assistant.

Conversation so far:
{context}

Clinical evidence (knowledge graph edges):
{evidence}

Task:
A) Update the caregiver plan based on current info (5-8 bullets).
B) If you cannot decide safely, ask at most 2 follow-up questions.
C) Include clear escalation thresholds if any are triggered.
D) Be consistent with prior steps and do not repeat generic safety lists.
"""
        out = llm.generate(prompt, temperature=0.2)

        # Pull questions (if any)
        followups = [ln.strip() for ln in out.splitlines() if ln.strip().endswith("?")]
        followups = followups[:2]

        if print_evidence:
            print("\nSTEP {step}: model plan / questions")
            print(out)
            if followups:
                print("\nSTEP {step}: extracted follow-ups:", followups)

        state["steps"].append({
            "step": step,
            "evidence_edges": len(edges),
            "evidence_text": evidence,
            "model_output": out,
            "followups": followups
        })

        # stop if no questions asked (converged) or urgent escalation recommended
        low = out.lower()
        if (not followups) or ("er now" in low) or ("go to the er" in low):
            state["final_answer"] = out
            break

        # For demo: append a plausible answer so you can see refinement.
        # Replace this line with actual user replies in your lab.
        state["conversation"].append("(demo) last pee was ~6 hours ago; he can sip ORS; belly pain diffuse, not localized; no blue lips.")

    if state["final_answer"] is None:
        state["final_answer"] = state["steps"][-1]["model_output"] if state["steps"] else ""

    return state


# -----------------------------
# 4) Controlled generation with a Cypher-style retrieval step
# -----------------------------
CONTROLLED_TEMPLATE = """
You are CareTrace, an after-hours pediatric triage assistant.

Output EXACTLY these sections, in order, with the labels exactly as shown:

1) What I think is going on (1-2 sentences)
2) What to do tonight (3-6 bullets)
3) What I need you to watch for (3-6 bullets)
4) Go now if ANY of these happen (3-6 bullets)

Constraints:
- Do not add any other sections.
- Do not provide long explanations.
- Do not provide medication dosing unless weight AND concentration are provided.
- Use only the retrieved evidence and rules below.
"""

def build_cypher_for_keywords(keywords: List[str], limit: int = 25) -> str:
    """
    Cypher-style query for a property graph with:
      (:Concept {id, name, type, aliases})
      -[:REL {why}]->
      (:Concept ...)
    You can map this to your actual Neo4j schema later.
    """
    # A simple pattern: match nodes whose name/aliases contain any keyword
    # then return edges adjacent to them.
    kw_list = [k.lower() for k in keywords if len(k) >= 3][:8]
    kw_json = json.dumps(kw_list)

    return f"""
WITH {kw_json} AS kws
MATCH (a:Concept)-[r]->(b:Concept)
WHERE any(k IN kws WHERE toLower(a.name) CONTAINS k OR any(al IN coalesce(a.aliases, []) WHERE toLower(al) CONTAINS k))
RETURN a.type AS a_type, a.name AS a_name, type(r) AS rel, b.type AS b_type, b.name AS b_name, coalesce(r.why, '') AS why
LIMIT {int(limit)}
""".strip()


def naive_keywords(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    toks = [t for t in text.split() if len(t) >= 3]
    # keep a few informative tokens
    stop = {"this", "that", "with", "have", "been", "from", "your", "what", "should", "tonight", "about", "really", "want"}
    toks = [t for t in toks if t not in stop]
    # de-dup preserve order
    seen = set()
    out = []
    for t in toks:
        if t not in seen:
            seen.add(t)
            out.append(t)
    return out[:10]


def neo4j_run_cypher(
    uri: str,
    user: str,
    password: str,
    cypher: str,
    database: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Optional: run Cypher against Neo4j AuraDB.
    Requires: pip install neo4j
    """
    from neo4j import GraphDatabase
    auth = (user, password)

    driver = GraphDatabase.driver(uri, auth=auth)
    try:
        with driver.session(database=database) as session:
            res = session.run(cypher)
            return [dict(r) for r in res]
    finally:
        driver.close()


def rows_to_evidence_text(rows: List[Dict[str, Any]]) -> str:
    if not rows:
        return "(no relevant KG evidence found)"
    lines = []
    for r in rows:
        lines.append(
            f"- ({r.get('a_type','?')}) {r.get('a_name','?')}  --{r.get('rel','?')}-->  ({r.get('b_type','?')}) {r.get('b_name','?')} :: {r.get('why','')}"
        )
    return "\n".join(lines)


def controlled_generation_verbose(
    llm: OllamaClient,
    kg: SimpleKG,
    user_question: str,
    use_neo4j: bool = False,
    neo4j_uri: str = "",
    neo4j_user: str = "",
    neo4j_password: str = "",
    neo4j_db: Optional[str] = None,
    print_debug: bool = True,
) -> str:
    """
    Shows:
      - the "Cypher" we would run
      - the evidence returned (Neo4j or in-memory fallback)
      - the final controlled answer
    """
    triggering_keywords = naive_keywords(user_question)

    cypher = build_cypher_for_keywords(triggering_keywords)

    if use_neo4j:
        rows = neo4j_run_cypher(neo4j_uri, neo4j_user, neo4j_password, cypher, database=neo4j_db)
        evidence = rows_to_evidence_text(rows)
    else:
        # fallback: in-memory retrieval (so the lab runs anywhere)
        edges = kg.retrieve_relevant_edges(user_question, max_edges=22)
        evidence = kg.edges_to_text(edges) if edges else "(no relevant KG evidence found)"

    rules = kg.get_rules()
    rules_text = "\n".join([f"- {r.name}" for r in rules]) if rules else "(no rules)"

    if print_debug:
        print("\n" + "=" * 70)
        print("CONTROLLED GENERATION: retrieval step")
        print("Keywords:", triggering_keywords)
        print("\nCypher (illustrative):\n", cypher)
        print("\nEvidence returned:\n", evidence)

    prompt = f"""
{CONTROLLED_TEMPLATE}

Evidence:
{evidence}

Rules checklist:
{rules_text}

User message:
{user_question}
"""
    out = llm.generate(prompt, temperature=0.2)
    if print_debug:
        print("\nCONTROLLED GENERATION: final answer\n")
        print(out)
    return out


# -----------------------------
# 5) Demo
# -----------------------------
def main_verbose():
    llm = OllamaClient(model="llama3.1")
    print("=== Ollama connectivity test ===")
    print(llm.generate("Return exactly: OK", temperature=0.0).strip())

    kg = SimpleKG(CARETRACE_KG)

    q1 = "It’s 11 PM. My 6-year-old has a fever, vomited once, stomach hurts, and he’s only peed once today. What should I do tonight?"
    q2 = "He can sip fluids but I am trying to avoid the ER if possible. What should I watch for?"
    q3 = "He has trouble breathing and looks blue around the lips."

    print("\n\n=== VERBOSE Method 2: KG-augmented reasoning (iterative refinement) ===")
    trace = verbose_kg_augmented_reasoning(llm, kg, conversation=[q1, q2], max_steps=3, print_evidence=True)
    print("\n\n=== FINAL (converged) ANSWER ===\n")
    print(trace["final_answer"])

    print("\n\n=== VERBOSE Method 3: Controlled generation with Cypher-style retrieval ===")
    _ = controlled_generation_verbose(llm, kg, q3, use_neo4j=False, print_debug=True)

if __name__ == "__main__":
    main_verbose()


=== Ollama connectivity test ===
OK


=== VERBOSE Method 2: KG-augmented reasoning (iterative refinement) ===

STEP 1: conversation context
- It’s 11 PM. My 6-year-old has a fever, vomited once, stomach hurts, and he’s only peed once today. What should I do tonight?
- He can sip fluids but I am trying to avoid the ER if possible. What should I watch for?

STEP {step}: retrieved KG evidence
- (Finding) Low urine output  --SUGGESTS-->  (Condition) Dehydration :: Low urine output can indicate dehydration.
- (Symptom) Vomiting  --SUGGESTS-->  (Condition) Viral gastroenteritis :: Vomiting can occur in viral gastroenteritis.
- (Symptom) Abdominal pain  --SUGGESTS-->  (Condition) Appendicitis :: Abdominal pain can be appendicitis depending on pattern.
- (Condition) Dehydration  --RECOMMENDS-->  (Action) Small frequent sips :: Small sips reduce vomiting risk.
- (Symptom) Fever  --RECOMMENDS-->  (Action) Acetaminophen (Tylenol) :: Acetaminophen helps discomfort/fever.
- (Symptom) Fever  --RECOM

In [7]:
# ============================================================
# (simple) KG Visualization Utility Function
# ============================================================


from pyvis.network import Network
from IPython.display import HTML, display

def show_kg_simple(kg, height="650px", width="100%", show_edge_labels=False):
    # ---- palettes ----
    type_colors = {
        "Symptom": "#4C78A8",
        "Sign": "#72B7B2",
        "Condition": "#F58518",
        "Medication": "#54A24B",
        "RedFlag": "#E45756",
        "Action": "#B279A2",
        "Rule": "#9D755D",
        "Guideline": "#FF9DA6",
        "Other": "#8E8E8E",
    }

    rel_colors = {
        "HAS_SYMPTOM": "#4C78A8",
        "INDICATES": "#F58518",
        "RED_FLAG_FOR": "#E45756",
        "RECOMMEND": "#54A24B",
        "AVOID": "#E45756",
        "ASK": "#72B7B2",
        "CHECK": "#B279A2",
        "SEE_ALSO": "#9D755D",
        "RELATED_TO": "#8E8E8E",
    }

    # ---- create network (inline) ----
    net = Network(height=height, width=width, directed=True, notebook=True)

    # ---- add nodes ----
    for node_id, node in kg.nodes.items():
        ntype = getattr(node, "type", "Other") or "Other"
        name = getattr(node, "name", str(node_id))
        color = type_colors.get(ntype, type_colors["Other"])
        title = f"{ntype}: {name}"

        net.add_node(
            node_id,
            label=name,
            title=title,
            color=color,
            shape="dot",
            size=14,
        )

    # ---- add edges ----
    for e in kg.edges:
        rel = getattr(e, "rel", "RELATED_TO")
        color = rel_colors.get(rel, rel_colors["RELATED_TO"])
        title = getattr(e, "why", rel)

        net.add_edge(
            e.src,
            e.dst,
            label=rel if show_edge_labels else "",
            title=title,
            color=color,
            arrows="to",
        )

    # ---- layout/physics + interaction ----
    net.set_options("""
    var options = {
      "interaction": {
        "hover": true,
        "dragNodes": true,
        "navigationButtons": true
      },
      "nodes": {
        "borderWidth": 1,
        "font": { "size": 14 }
      },
      "edges": {
        "smooth": false,
        "font": { "size": 11, "align": "middle" }
      },
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -60,
          "centralGravity": 0.01,
          "springLength": 150,
          "springConstant": 0.08
        },
        "minVelocity": 0.75,
        "solver": "forceAtlas2Based"
      }
    }
    """)

    # ---- render inline ----
    html = net.generate_html()
    display(HTML(html))

# Usage:
show_kg_simple(kg, show_edge_labels=False)
